<a href="https://colab.research.google.com/github/Pranavi2709/FaceX-Zoo/blob/main/Hypertuning_Bio-InspiredOptimisation/PSO_Ravdess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/tulika-jha/Speech-Emotion-Recognition.git"

Cloning into 'Speech-Emotion-Recognition'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 14 (delta 6), reused 14 (delta 6), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [2]:
#select top 177 features of the 652 features available in Revdess
!python "/content/Speech-Emotion-Recognition/selection.py"

shape of MFCC features ->  (1440, 360)
(1440, 72)  <- MFCC
shape of LFCC features ->  (1440, 234)
(1440, 47)  <- LFCC
shape before feature selection ->  (1440, 652)
concatenated........
(1440, 177)


In [3]:
#Data Pre-processing

# Import Required Modules.
import json
from statistics import mean, stdev

import imblearn
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold,KFold
import numpy as np
from sklearn.ensemble import RandomForestClassifier

#open extracted features
from sklearn.preprocessing import StandardScaler

with open("/content/Speech-Emotion-Recognition/features_652.json", "r") as f:
    data_dict = json.load(f)

data_with_labels = data_dict["0"]
data = []
labels = []

#populating data matrix
for i in range(len(data_with_labels)):
    data.append(data_with_labels[i][0])
    labels.append(data_with_labels[i][1])

data = np.load("X_diff.npy")
data = np.asarray(data)


#oversampling minority class
oversample = imblearn.over_sampling.SMOTE()
data, labels = oversample.fit_resample(data, labels)


# Feature Scaling for input features.
sc = StandardScaler()
x_scaled = sc.fit_transform(data)

y = np.asarray(labels)
print(x_scaled,labels)

[[-0.51935142 -0.36156201 -0.30005866 ... -0.19801925 -0.61896723
   0.03933657]
 [-0.17149667 -0.01802615 -0.16284518 ... -0.47645087 -2.57362613
   0.29627542]
 [-0.53544582 -0.40415898 -0.33219629 ...  0.24936963  0.11821196
   0.17678383]
 ...
 [-0.97362885 -0.76904226 -0.72709691 ... -0.80957694  1.36973313
  -1.04841077]
 [-0.96980835 -1.00217045 -0.87356991 ... -0.81140676 -0.2608925
   0.0907645 ]
 [-1.20159901 -0.81048613 -0.92089896 ... -0.85267172 -0.30163777
  -0.20781613]] ['neutral', 'neutral', 'neutral', 'neutral', 'calm', 'calm', 'calm', 'calm', 'calm', 'calm', 'calm', 'calm', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'fearful', 'fearful', 'fearful', 'fearful', 'fearful', 'fearful', 'fearful', 'fearful', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'surprised', 'surprised'

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats



In [5]:
!pip install Optunity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 7.1 MB/s 
  Created wheel for Optunity: filename=Optunity-1.1.1-py3-none-any.whl size=72031 sha256=12856028b9bd2cecea2d607ccd178f56a3da845370a77c4487779aa65500c51e
  Stored in directory: /root/.cache/pip/wheels/d1/03/a8/c725421da03e2bb8b33fb20b7159727fb1d47806f0ae844a91
Successfully built Optunity


In [27]:
#Random Forest
import optunity
import optunity.metrics
data=x_scaled
labels=y.tolist()
search = {
    'n_estimators': [10, 100],
    'max_features': [1, 64],
    'max_depth': [5,50],
    "min_samples_split":[2,11],
    "min_samples_leaf":[1,11],
    "criterion":[0,1]
         }
# Define the objective function
@optunity.cross_validated(x=data, y=labels, num_folds=3)
def performance(x_train, y_train, x_test, y_test,n_estimators=None, max_features=None,max_depth=None,min_samples_split=None,min_samples_leaf=None,criterion=None):
    # fit the model
    if criterion<0.5:
        cri='gini'
    else:
        cri='entropy'
    model = RandomForestClassifier(n_estimators=int(n_estimators),
                                   max_features=int(max_features),
                                   max_depth=int(max_depth),
                                   min_samples_split=int(min_samples_split),
                                   min_samples_leaf=int(min_samples_leaf),
                                   criterion=cri,
                                  )
    #predictions = model.predict(x_test)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scores=np.mean(cross_val_score(model,x_scaled , y, cv=skf, n_jobs=-1,
                                    scoring="accuracy"))
    #return optunity.metrics.roc_auc(y_test, predictions, positive=True)
    return scores#optunity.metrics.accuracy(y_test, predictions)

optimal_configuration, info, _ = optunity.maximize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("Accuracy:"+ str(info.optimum))
#evals-50 {'n_estimators': 89.89006328219013, 'max_features': 43.78584899726307, 'max_depth': 35.799249011550614, 'min_samples_split': 10.885061159272416, 'min_samples_leaf': 1.3125867049429494, 'criterion': 0.6941009103672451}
#Accuracy:0.4631076388888889
#evals-10{'n_estimators': 78.41324620109611, 'max_features': 46.50699073112609, 'max_depth': 47.876639131183815, 'min_samples_split': 6.39541015625, 'min_samples_leaf': 1.7041015625, 'criterion': 0.4773752169056914}
#Accuracy:0.46723090277777785
#{'n_estimators': 84.5751953125, 'max_features': 55.29443359375, 'max_depth': 44.88037109375, 'min_samples_split': 10.11669921875, 'min_samples_leaf': 2.1474609375, 'criterion': 0.82861328125}
#Accuracy:0.4631076388888889
#skfold,evals=20

{'n_estimators': 78.642578125, 'max_features': 53.6025390625, 'max_depth': 38.2666015625, 'min_samples_split': 4.6806640625, 'min_samples_leaf': 1.419921875, 'criterion': 0.9833984375}
Accuracy:0.6334719746484452


In [28]:
#SVM
import optunity
import optunity.metrics

data=x_scaled
labels=y.tolist()

search = {
    'C': (0,100),
    'kernel':[0,6]
         }
@optunity.cross_validated(x=data, y=labels, num_folds=3)
def performance(x_train, y_train, x_test, y_test,C=None,kernel=None):
    # fit the model
    if kernel<1:
        ke='linear'
    elif kernel<2:
        ke='poly'
    elif kernel<3:
        ke='rbf'
    else:
        ke='sigmoid'
    model = SVC(C=float(C),
                kernel=ke
                                  )
    #predictions = model.predict(x_test)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scores=np.mean(cross_val_score(model, x_scaled, y, cv=skf, n_jobs=-1,
                                    scoring="accuracy"))
    #return optunity.metrics.roc_auc(y_test, predictions, positive=True)
    return scores#optunity.metrics.accuracy(y_test, predictions)

optimal_configuration, info, _ = optunity.maximize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("Accuracy:"+ str(info.optimum))

#skf,evals=20,{'C': 41.650390625, 'kernel': 2.2998046875}
#Accuracy:0.78781512605042

{'C': 41.650390625, 'kernel': 2.2998046875}
Accuracy:0.78781512605042


In [35]:
#KNN
import optunity
import optunity.metrics

data=x_scaled
labels=y.tolist()

search = {
    'n_neighbors': [1, 20],
         }
@optunity.cross_validated(x=data, y=labels, num_folds=3)
def performance(x_train, y_train, x_test, y_test,n_neighbors=None):
    # fit the model
    model = KNeighborsClassifier(n_neighbors=int(n_neighbors))
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)                             
    scores=np.mean(cross_val_score(model, x_scaled, y, cv= skf, n_jobs=-1,
                                    scoring="accuracy"))
    return scores

optimal_configuration, info, _ = optunity.maximize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=30,
                                                   **search
                                                  )
print(optimal_configuration)
print("Accuracy:"+ str(info.optimum))
#{'n_neighbors': 1.25048828125}
#Accuracy:0.6536414565826332

{'n_neighbors': 1.482421875}
Accuracy:0.6536414565826332
